# Код для парсинга профилей и постов LinkedIn

In [ ]:
import time
import configparser
import random

import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities

**Загружаем конфиг**

In [ ]:
# путь к файлу расширения для Chrome "Доступ к LinkedIn"
EXTENSION_PATH = '1.5_0.crx'

# конфиг
CFG_FILE = 'parser.ini'

"""
файл конфигурации необходимо предварительно создать,
формат файла parser.ini:
[LINKEDIN]
USER_LOGIN = эл_почта_без_кавычек
USER_PASSWORD = пароль
""";

# загружаем данные из конфига
conf = configparser.ConfigParser()
try:
    conf.read(CFG_FILE)
    USER_LOGIN = conf['LINKEDIN']['USER_LOGIN']
    USER_PASSWORD = conf['LINKEDIN']['USER_PASSWORD']
except:
    print(f'Не удалось прочитать файл конфигурации: {CFG_FILE}')
    quit() # нужен только при переносе кода в .py

**Общие процедуры и функции**

In [ ]:
# получить и отобразить информацию профиля
def get_and_print_profile_info(driver, profile_url):
    driver.get(profile_url)        # открываем ссылку profile_url

    # извлекаем код страницы
    src = driver.page_source

    # предеаем код страницы в парсер
    soup = BeautifulSoup(src, 'lxml')
    #soup.prettify()

    # извлекаем HTML содержаший имя и заголовок
    intro = soup.find('div', {'class': 'pv-text-details__left-panel'})

    #print(f'''{intro}''') # вывод содержимого для контроля

    # в случае ошибки попробуйте изменить используемые здесь теги
    name_loc = intro.find("h1")

    # получаем имя
    name = name_loc.get_text().strip()

    # заголовок, обычно тут пишут где работает или специальность
    works_at_loc = intro.find("div", {'class': 'text-body-medium'})

    # получаем компанию или специальность
    works_at = works_at_loc.get_text().strip()

    print(f'''
    Name --> {name},
    Works At --> {works_at}
    ''')

    # суффикс к url профиля для отображения всех публикаций пользователя
    POSTS_URL_SUFFIX = 'recent-activity/all/'

    time.sleep(random.uniform(0.5, 1.5))

    # текущий url профиля пользователя
    cur_profile_url = driver.current_url
    print(f'''
    Profile URL --> {cur_profile_url}
    ''')

    # получим все публикации пользователя
    get_and_print_user_posts(driver, cur_profile_url + POSTS_URL_SUFFIX)

In [ ]:
# прокрутка страницы, для подгрузки динамического контента
def get_scrolled_page(driver, num_scrolls=15, pause_time=1.5):
    """
    Функция прокручивает страницу, загруженную в экземпляр driver,
    num_scrolls раз, с pause_time паузами между прокрутками.
    Возвращает код страницы.
    """
    # получим высоту прокрутки
    last_height = driver.execute_script("return document.body.scrollHeight")

    for i in range(num_scrolls):
        # прокрутка вниз
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # ожидаем загрузку страницы
        time.sleep(random.uniform(pause_time, pause_time+1))

        # новая высота прокрутки и сравниваем с последней высотой прокрутки
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    return driver

In [ ]:
# получить и отобразить все публикации из профиля
def get_and_print_user_posts(driver, posts_url):
    driver.get(posts_url)

    # получаем код проскроленой страницы
    src = get_scrolled_page(driver).page_source

    # передаем код страницы в парсер
    soup = BeautifulSoup(src, 'lxml')
    #soup.prettify()

    # получаем список постов
    posts = soup.find_all('li', class_='profile-creator-shared-feed-update__container')

    print(f'''
    Number of posts: {len(posts)}
    ''')
    for post_src in posts:
        # блок с текстом поста
#        post_text_div = post_src.find('div', {'class': 'feed-shared-update-v2__description-wrapper mr2'})
        post_text_div = post_src.find('div', {'class': 'feed-shared-update-v2__description-wrapper'})

        if post_text_div is not None:
            # текст поста
            post_text = post_text_div.find('span', {'dir': 'ltr'})
        else:
            post_text = None

        # если пост содержит текст
        if post_text is not None:
            post_text = post_text.get_text().strip()
            print(f'''
            Post text: {post_text}
            ''')

        # число реакций на пост
        reaction_cnt = post_src.find('span', {'class': 'social-details-social-counts__reactions-count'})

        # если число реакций записано в виде текста, то используем другое имя класса
        if reaction_cnt is None:
            reaction_cnt = post_src.find('span', {'class': 'social-details-social-counts__social-proof-text'})

        if reaction_cnt is not None:
            reaction_cnt = reaction_cnt.get_text().strip()
            print(f'''
            Reactions: {reaction_cnt}
            ''')

    return

In [ ]:
# формируем запрос на поиск людей по ключевым словам
def search_people_url(keywords, tags, page_num=1):
    """
    Функция на вход получает ключевые слова,
    список тем публикаций для поиска и номер страницы.
    Возвращает url для запроса страницы.
    """
    # преобразуем теги в строку
    tags_str = str(tags).replace(" ", "").replace("'", '"')
    
    # формируем строку запроса
    search_url = 'https://www.linkedin.com/search/results/people/'
    search_url += f'?keywords={keywords}'
    search_url += '&origin=FACETED_SEARCH'
    search_url += f'&page={page_num}'
    search_url += '&profileLanguage=["ru"]'
    # темы публикаций (хештеги)
    search_url += f'&talksAbout={tags_str}'
    
    return search_url

In [ ]:
# получаем список профилей на странице
def get_profiles(driver):
    """
    Функция получает драйвер открытой страницы,
    ищет ссылки на доступные профили пользователей и возвращает
    список id пользователей.
    """
    # список найденных профилей
    profiles = []

    # ищем на странице ссылки на профили
    finded_profiles = driver.find_elements(
        By.CSS_SELECTOR, "span.entity-result__title-text a.app-aware-link"
    )
    for profile in finded_profiles:
        # получаем url на профиль пользователя
        url = profile.get_attribute("href")
        # если url ссылается на доступный профиль
        if 'linkedin.com/in' in url:
            # оставляем только id профиля
            profile_id = url.split('?')[0].split('/in/')[1]
            # добавляем id в список
            profiles.append(profile_id)

    # избавляемся от дублей, если вдруг появятся
    profiles = list(set(profiles))
    return profiles

**Создаем и запускаем браузер**

In [ ]:
# подключаем расширение к драйверу
options = webdriver.ChromeOptions()
options.add_extension(EXTENSION_PATH)

# меняем стратегию - ждать, пока свойство document.readyState примет значение interactive
options.page_load_strategy = 'eager'

# запускаем Chrome с расширением
driver = webdriver.Chrome(options=options)

**Вход в LinkedIn**

In [ ]:
# открываем страницу входа linkedIn, необходимо отключить двухфакторную аутонтификацию
driver.get("https://linkedin.com/uas/login")

# ожидаем загрузку страницы
time.sleep(4)

# поле ввода имени пользователя
username = driver.find_element(By.ID, "username")

# вводим свой Email
username.send_keys(USER_LOGIN)

# поле ввода пароля
pword = driver.find_element(By.ID, "password")

# вводим пароль
pword.send_keys(USER_PASSWORD)

# нажимаем кнопку Войти
# Формат (синтаксис) написания XPath --> //tagname[@attribute='value']
driver.find_element(By.XPATH, "//button[@type='submit']").click()

**Параметры парсинга**

In [ ]:
# теги, темы публикаций

#KEYWORDS = 'разработка по'
#TAGS = ['softwaredevelopment', 'webdevelopment', 'startup', 'it', 'design']
#CSV_FILE_NAME = 'profiles_id_1.csv'

#KEYWORDS = 'devops'
#TAGS = ['devops', 'aws', 'python', 'cloud', 'kubernetes']
#CSV_FILE_NAME = 'profiles_id_2.csv'

#KEYWORDS = 'data science'
#TAGS = ['datascience', 'machinelearning', 'ai', 'artificialintelligence', 'dataanalytics']
#CSV_FILE_NAME = 'profiles_id_3.csv'

#KEYWORDS = 'project management'
#TAGS = ['projectmanagement', 'business', 'agile', 'scrum', 'it']
#CSV_FILE_NAME = 'profiles_id_4.csv'

#KEYWORDS = 'design ui ux'
#TAGS = ['design', 'webdesign', 'ux', 'ui', 'uxdesign', 'uidesign']
#CSV_FILE_NAME = 'profiles_id_5.csv'

KEYWORDS = 'анализ данных'
TAGS = ['marketing', 'ai', 'datascience', 'productmanagement', 'machinelearning']
CSV_FILE_NAME = 'profiles_id_6.csv'

# число страниц для парсинга, в бесплатном аккаунте доступно не более 100
NUM_PAGES = 100

**Собираем ID пользователей**

In [ ]:
# пустой датафрейм для id пользователей
df = pd.DataFrame(columns=['id'])

for page_num in range(1, NUM_PAGES+1):
    
    # выводим номер страницы, в случае сбоя можно будет начать новый парсинг с нее
    print(page_num, end='  ')
    
    # формируем url запроса
    people_url = search_people_url(KEYWORDS, TAGS, page_num=page_num)
    
    # запрашиваем и открываем страницу
    driver.get(people_url)
    
    # получаем и добавляем список найденных id профилей на странице
    profiles_id = get_profiles(driver)
    
    # добавляем данные в датафрейм
    df = pd.concat(
        [df, pd.DataFrame({'id': profiles_id})]
    ).reset_index(drop=True)
    
    # сохраняем в CSV
    df.to_csv(CSV_FILE_NAME)

    time.sleep(random.uniform(5, 10))

**Собираем все id в один датафрейм**

In [ ]:
# список файлов c id пользователей
list_csv_files = [
    'profiles_id_1.csv',
    'profiles_id_2.csv',
    'profiles_id_3.csv',
    'profiles_id_4.csv',
    'profiles_id_5.csv',
    'profiles_id_6.csv',
]

# очистим данные
df = pd.DataFrame(columns=['id'])

# соберем все файлы в один DF
for csv_file in list_csv_files:
    df = pd.concat(
        [df, pd.read_csv(csv_file, index_col=0)]
    ).reset_index(drop=True)
    
# удаляем дубли
df = df.drop_duplicates()

In [ ]:
print('Всего профилей:', len(df))

In [ ]:
# закрываем браузер
driver.quit()

Какую информацию нам нужно получить.

Из профилей пользователей:

- Ссылка
- Имя
- Возраст (нет), можно попробовать Дата присоединения
- Теги
- Локация
- Компания
- Должность
- Направление, Опыт

И публикаций:

- Текст поста
- Количество лайков
- Количество комментариев
